# Imports

In [1]:
from pytube import YouTube
from pydub import AudioSegment
import os
import whisper
import wave
from spleeter.separator import Separator
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# Functions

In [2]:
def transform_audio_format(input_file_path, output_dir_path):
    """
    Transforms an unspecified audio file into .wav format with mono channels,
    a 2 byte channel width, and 16000 Hz.

    :param input_file_path: a string path of the file to be formatted
    :param output_dir_path: a string path of the directory to save the new .wav
    :returns: N/A
    """
    audio = AudioSegment.from_file(input_file_path)
    
    audio = audio.set_channels(1)
    audio = audio.set_sample_width(2)
    audio = audio.set_frame_rate(16000)

    audio.export(output_dir_path, format="wav")


def verify_audio_format(audio_file_path):
    """
    Verifies that an audio file has the format of mono, 2 byte channel width,
    16000 Hz, and .wav extension.

    :param audio_file_path: a string path of the file to be verified
    :returns: N/A
    """
    with wave.open(audio_file_path, 'rb') as file:
        if (audio_file_path.lower().endswith('.wav') and 
            file.getnchannels() == 1 and
            file.getsampwidth() == 2 and
            file.getframerate() == 16000):
            print('\nFile is in correct format.\n')
            return True
        else:
            print('\nFile is not in correct format.\n')
            return False


def download_youtube_audio(url, output_dir_path):
    """
    Downloads the audio of a YouTube URL in .mp4 format.

    :param url: a string of the YouTube URL
    :param output_dir_path: a string path of the directory to save the new .wav
    :returns: a string path to the .mp4 of the YouTube video
    """
    return YouTube(url).streams.filter(only_audio=True).first().download(
        output_path=output_dir_path
    )


def transcribe_audio(vocal_file_path):
    """
    Transctibes the vocal 

    :param url: a string of the YouTube URL
    :param output_dir_path: a string path of the directory to save the new .wav
    :returns: a string path to the .mp4 of the YouTube video
    """
    if not verify_audio_format(vocal_file_path):
        raise Exception('Cannot transcribe audio with invalid file format.')

    base_model = whisper.load_model('medium')

    try:
        transcription = base_model.transcribe(vocal_file_path)
        return transcription['text']
    except Exception as e:
        print('Unable to transcribe audio.')
        print(e)

# Download YouTube Audio and Convert the Format

In [3]:
url = 'https://www.youtube.com/watch?v=pat2-f0BXCg' # Asthenia - blink-182
# url = 'https://www.youtube.com/watch?v=FVdjZYfDuLE' # Wonderwall - Oasis

# create the output directory to house audio file downloads
output_dir_path = 'Output/'
os.makedirs(output_dir_path, exist_ok=True)

# convert and download an audio file (in .mp4 format) from a youtube url
mp4_file_path = download_youtube_audio(url, output_dir_path)
try: print('DOWNLOADED FILE:', mp4_file_path.split('/')[-1])
except: print('DOWNLOADED FILE:', mp4_file_path.split('\\')[-1])

# get the name of the file without file path or file extension
file_name, _ = os.path.splitext(os.path.basename(mp4_file_path))

# set the full path of the .wav file for the audio
wav_file_path = os.path.join(output_dir_path, file_name + '.wav')

# convert the file from .mp4 to .wav
transform_audio_format(mp4_file_path, wav_file_path)
print('CONVERTED FILE:', wav_file_path)

DOWNLOADED FILE: Wonderwall.mp4
CONVERTED FILE: Output/Wonderwall.wav


# Separate the Vocals from the music

In [4]:
separator = Separator('spleeter:2stems')
separator.separate_to_file(wav_file_path, 'Output/');

INFO:spleeter:File Output/Wonderwall/vocals.wav written succesfully
INFO:spleeter:File Output/Wonderwall/accompaniment.wav written succesfully


In [5]:
vocals_file_path = f"Output/{file_name}/vocals.wav"
accompaniment_file_path = f"Output/{file_name}/accompaniment.wav"

# Transcribe the Vocal Track

In [6]:
transform_audio_format(vocals_file_path, vocals_file_path)

transcription = transcribe_audio(vocals_file_path)
print(transcription)


File is in correct format.



100%|█████████████████████████████████████| 1.42G/1.42G [12:35<00:00, 2.02MiB/s]
/Users/mattkarenbauer/anaconda3/envs/spleeter/lib/python3.8/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Today is gonna be the day that they're gonna throw it back to you I don't believe that anybody feels the way I do about you now Backbeat the word is on the street that the fire in your heart is out I'm sure you've heard it all before but you never really had a doubt I don't believe that anybody feels the way I do about you now And all the roads we have to walk are winding And all the lights I need to stare at blinding There are many things that I would like to say to you but I don't know how Cos maybe you're gonna be the one that saves me And after all you're my Wonderwall Today is gonna be the day that they're gonna throw it back to you By now you should have somehow realized what you're not to do I don't believe that anybody feels the way I do about you now And all the roads that lead you there were winding And all the lights that light the way of blinding There are many things that I would like to say to you but I don't know how Cos maybe you're gonna be the one that saves me And a

Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/mattkarenbauer/anaconda3/envs/spleeter/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/mattkarenbauer/anaconda3/envs/spleeter/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mattkarenbauer/anaconda3/envs/spleeter/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/mattkarenbauer/anaconda3/envs/spleeter/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  File "/Users/mattkarenbauer/anaconda3/envs/spleeter/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process SpawnPoolWorker-8:
Traceback (most recent call last):
  File "/Users/mattkarenbauer/anaconda3/envs/spleeter/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
